In [1]:
import os
os.environ['GRB_LICENSE_FILE'] = r'C:\Users\AKSHAY\gurobi.lic'

In [2]:
from gurobipy import Model, GRB, quicksum

In [3]:
# Input Data
stellar_crystals = [(10, 20, 100), (40, 50, 150), (25, 35, 200), (80, 50, 300)]
void_mines = [(15, 25, 50), (70, 60, 75), (30, 200, 60)]
n = len(stellar_crystals)
m = len(void_mines)

In [4]:
# Create Model
model = Model("Polygon_Containment")

# Number of polygon vertices
num_vertices = 1000

Set parameter Username
Set parameter LicenseID to value 2616846
Academic license - for non-commercial use only - expires 2026-01-31


In [5]:
# Decision Variables
x = model.addVars(num_vertices, vtype = GRB.CONTINUOUS, lb = 0, ub = 10000, name = "X")  # X-coordinates of polygon
y = model.addVars(num_vertices, vtype = GRB.CONTINUOUS, lb = 0, ub = 10000, name = "Y")  # Y-coordinates of polygon
z = model.addVars(n, vtype = GRB.BINARY, name = "Stellar_Inside")  # Stellar Crystal inclusion
w = model.addVars(m, vtype = GRB.BINARY, name = "Void_Inside")  # Void Mine inclusion

In [6]:
# Objective Function: Maximize net value
model.setObjective(
    quicksum(stellar_crystals[i][2] * z[i] for i in range(n)) - quicksum(void_mines[j][2] * w[j] for j in range(m)),
    GRB.MAXIMIZE
)

In [7]:
# Constraint 1: Closed polygon

model.addConstr(x[num_vertices - 1] == x[0])
model.addConstr(y[num_vertices - 1] == y[0])

<gurobi.Constr *Awaiting Model Update*>

In [8]:
# Constraint 2: Ensure edges are horizontal or vertical
big_M = 10000  # Large constant for Big-M constraints
for v in range(num_vertices - 1):
    is_horizontal = model.addVar(vtype=GRB.BINARY, name=f"IsHorizontal_{v}")  # 1 if horizontal, 0 if vertical

    # Ensure movement in x OR y direction
    model.addConstr(x[v+1] - x[v] <= big_M * is_horizontal)
    model.addConstr(y[v+1] - y[v] <= big_M * (1 - is_horizontal))

    # Ensure actual movement happens
    model.addConstr((x[v+1] - x[v]) + (y[v+1] - y[v]) >= 1, f"force_movement_{v}")

In [9]:
# Constraint 3: Non-Self-Intersection using Orientation Function
for i in range(num_vertices - 2):
    for j in range(i + 2, num_vertices - 1):  # Ensure we only compare non-adjacent edges
        if i == 0 and j == num_vertices - 2:  # Ignore first and last edge connection
            continue
        
        # Create auxiliary variables for orientation values
        o1 = model.addVar(vtype=GRB.CONTINUOUS, name=f"Orientation_o1_{i}_{j}")
        o2 = model.addVar(vtype=GRB.CONTINUOUS, name=f"Orientation_o2_{i}_{j}")
        o3 = model.addVar(vtype=GRB.CONTINUOUS, name=f"Orientation_o3_{i}_{j}")
        o4 = model.addVar(vtype=GRB.CONTINUOUS, name=f"Orientation_o4_{i}_{j}")

        # Compute orientation values using constraints
        model.addConstr(o1 == (y[j] - y[i]) * (x[i+1] - x[i]) - (x[j] - x[i]) * (y[i+1] - y[i]))
        model.addConstr(o2 == (y[j+1] - y[i]) * (x[i+1] - x[i]) - (x[j+1] - x[i]) * (y[i+1] - y[i]))
        model.addConstr(o3 == (y[i] - y[j]) * (x[j+1] - x[j]) - (x[i] - x[j]) * (y[j+1] - y[j]))
        model.addConstr(o4 == (y[i+1] - y[j]) * (x[j+1] - x[j]) - (x[i+1] - x[j]) * (y[j+1] - y[j]))

        # Binary variables to enforce non-self-intersection
        b1 = model.addVar(vtype=GRB.BINARY, name=f"b1_{i}_{j}")
        b2 = model.addVar(vtype=GRB.BINARY, name=f"b2_{i}_{j}")

        # Big-M constraints to enforce correct intersection conditions
        model.addConstr(o1 <= big_M * b1)
        model.addConstr(o2 >= -big_M * (1 - b1))

        model.addConstr(o3 <= big_M * b2)
        model.addConstr(o4 >= -big_M * (1 - b2))

        # Ensure that at least one condition holds
        model.addConstr(b1 + b2 >= 1, f"no_intersection_{i}_{j}")


In [10]:
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2490509 rows, 2988018 columns and 4985014 nonzeros
Model fingerprint: 0x234c735c
Model has 1990008 quadratic constraints
Variable types: 1992008 continuous, 996010 integer (996010 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [5e+01, 3e+02]
  Bounds range     [1e+00, 1e+04]
  RHS range        [1e+00, 1e+04]
Presolve removed 2487512 rows and 1990017 columns (presolve time = 8s)...
Presolve removed 2487512 rows and 1990017 columns (presolve time = 12s)...
Presolve removed 2487512 rows and 1990017 columns (presolve time = 18s)...
Presolve removed 2487512 rows and 1990017 columns
Presolve time: 18.34s
Presolved: 4986009 ro

GurobiError: Out of memory

In [12]:
# Output Results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Net Value: {model.objVal}")
    
    points = []
    for v in range(num_vertices):
        x_val = round(x[v].x)
        y_val = round(y[v].x)
        points.append((x_val, y_val))
    
    # Ensure it is a closed polygon
    points.append(points[0])

    print("\nFinal Polygon Vertices:")
    for p in points:
        print(p)
    
    # Optional: Plot the polygon
    import matplotlib.pyplot as plt

    px, py = zip(*points)
    
    plt.figure(figsize=(6,6))
    plt.plot(px, py, marker="o", linestyle="-", color="blue", markersize=4)
    plt.xlim(0, 10000)
    plt.ylim(0, 10000)
    plt.grid(True)
    plt.title("Optimized Non-Self-Intersecting Polygon")
    plt.show()